In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
import tensorflow as tf
from sklearn.metrics import classification_report

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
file_path = '/content/drive/MyDrive/ML/Amazon_Unlocked_Mobile.csv'
data = pd.read_csv(file_path)

In [6]:
data = data.dropna()

In [7]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = " ".join([word for word in text.split() if word not in stopwords.words('english')])
    return text

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
data['CleanedReviews'] = data['Reviews'].apply(clean_text)

In [10]:
#Label encoding
def sentiment(rating):
    if rating < 3:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

data['Sentiment'] = data['Rating'].apply(sentiment)

In [11]:
# Split dataset into training and testing sets
X = data['CleanedReviews']
y = data['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Tokenization and padding
max_length = 128
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_encoded = tokenizer(list(X_train), padding=True, truncation=True, max_length=max_length, return_tensors='tf').data
X_test_encoded = tokenizer(list(X_test), padding=True, truncation=True, max_length=max_length, return_tensors='tf').data

In [13]:
# Custom model with DistilBert layer
class SentimentModel(tf.keras.Model):
    def __init__(self):
        super(SentimentModel, self).__init__()
        self.distilbert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dense = tf.keras.layers.Dense(3, activation='softmax')

    def call(self, inputs):
        output = self.distilbert(inputs)
        pooled_output = output[0][:, 0, :]
        return self.dense(pooled_output)

In [14]:
model = SentimentModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [15]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [16]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train_encoded, y_train_encoded, epochs=5, batch_size=16, validation_split=0.2, callbacks=[early_stop])

Epoch 1/5
13374/13374 [==============================] - 3102s 229ms/step - loss: 0.3481 - accuracy: 0.8766 - val_loss: 0.2871 - val_accuracy: 0.9013
Epoch 2/5
13374/13374 [==============================] - 2985s 223ms/step - loss: 0.2408 - accuracy: 0.9184 - val_loss: 0.2544 - val_accuracy: 0.9158
Epoch 3/5
13374/13374 [==============================] - 2983s 223ms/step - loss: 0.1782 - accuracy: 0.9424 - val_loss: 0.2472 - val_accuracy: 0.9245
Epoch 4/5
13374/13374 [==============================] - 2982s 223ms/step - loss: 0.1384 - accuracy: 0.9566 - val_loss: 0.2373 - val_accuracy: 0.9287
Epoch 5/5
13374/13374 [==============================] - 2964s 222ms/step - loss: 0.1144 - accuracy: 0.9648 - val_loss: 0.2783 - val_accuracy: 0.9291


In [17]:
y_pred = model.predict(X_test_encoded)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Classification Report:\n", classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))

2090/2090 [==============================] - 289s 137ms/step
Classification Report:
               precision    recall  f1-score   support

    negative       0.91      0.90      0.91     15609
     neutral       0.70      0.65      0.67      5228
    positive       0.96      0.97      0.97     46030

    accuracy                           0.93     66867
   macro avg       0.86      0.84      0.85     66867
weighted avg       0.93      0.93      0.93     66867

